# <h1 align=center>Toranto Data</h1>

by: Nishad Thakur

Notebook has 3 parts separated by 3 questions

### 1) Import DATA - Web scraping

In [1]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=be5f6928dcc6df7be847f7da4c11818a92d198a6fe851bbd42de566b4ffa84b0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_data=requests.get(url).text
soup = BeautifulSoup(wiki_data, 'html5lib') #soup for getting data
table_contents=[]
table=soup.find('table')
len(table)

2

In [3]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)   #since its not vertical table we need to clean data

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_postcode.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Shape of data frame
df_postcode.shape

(103, 3)

Answer of question 1 is (103,3)

## latitudes and longitudes for various neighbourhoods in Canada

In [9]:
!pip install geocoder  #Geocoder for lattitude and longitude data

     |████████████████████████████████| 98 kB 8.1 MB/s  eta 0:00:01


In [10]:
import geocoder # import geocoder

In [11]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
print("The shape of our wiki data is: ", df_postcode.shape)
print("the shape of our csv data is: ", data.shape)

The shape of our wiki data is:  (103, 3)
the shape of our csv data is:  (103, 3)


In [15]:
df_postcode.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [14]:
data.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [18]:
combined_data = df_postcode.join(data.set_index('Postal Code'), on='PostalCode', how='inner')
combined_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Question 2 we got 103 rows with geo data

## 3) Cluster

Drawing inspiration from the previous lab where we cluster the neighbourhood of NYC, We cluster Toronto based on the similarities of the venues categories using Kmeans clustering and Foursquare API.

In [19]:
from geopy.geocoders import Nominatim

In [20]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [21]:
import folium #visualization

In [24]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighborhood in zip(combined_data['Latitude'], combined_data['Longitude'], combined_data['Borough'], combined_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

In [25]:
CLIENT_ID = 'O1LW2JUKJUJYE2ODTTVL2UPKT0URN45ZPDCHKIS0KY3OGMDC' 
CLIENT_SECRET = 'RMTYY4NRDXGY04EQNZFI2IQ0SFNICO11XFLPYJB4OJQDLZTU'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)  

Your credentails:
CLIENT_ID: O1LW2JUKJUJYE2ODTTVL2UPKT0URN45ZPDCHKIS0KY3OGMDC
CLIENT_SECRET:RMTYY4NRDXGY04EQNZFI2IQ0SFNICO11XFLPYJB4OJQDLZTU


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
venues_in_toronto = getNearbyVenues(combined_data['Neighborhood'], combined_data['Latitude'], combined_data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [30]:
venues_in_toronto.shape

(1332, 5)

So we have 1332 records and 5 columns. Checking sample data

In [31]:
venues_in_toronto.head()

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,Bank


Checking the Venues based on Neighbourhood

In [34]:
venues_in_toronto.groupby('Neighborhood').head()

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,Bank
...,...,...,...,...,...
1327,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Economy Rent A Car,Rental Car Location
1328,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Logistics Distribution,Bar
1329,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Saand Rexdale,Drugstore
1330,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,PC Garden,Garden Center


So there are 433 records for each neighbourhood.

In [35]:
venues_in_toronto.groupby('Venue Category').max()

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Ardene Shoes Outlet
Airport,Downsview East,43.737473,-79.394420,Toronto Downsview Airport (YZD)
Airport Food Court,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Billy Bishop Café
Airport Lounge,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Porter Lounge
Airport Service,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,ORNGE - Toronto Air Base
...,...,...,...,...
Warehouse Store,Thorncliffe Park,43.705369,-79.349372,Costco
Wine Bar,"Toronto Dominion Centre, Design Exchange",43.653206,-79.379817,The National Club
Wings Joint,Ontario Provincial Government,43.662301,-79.389494,Wingporium


There are around 234 different types of Venue Categories

In [36]:
toronto_venue_cat = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding the neighbourhood to the encoded dataframe

In [38]:
toronto_venue_cat['Neighborhood'] = venues_in_toronto['Neighborhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_venue_cat.columns[-1]] + list(toronto_venue_cat.columns[:-1])
toronto_venue_cat = toronto_venue_cat[fixed_columns]

toronto_venue_cat.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_grouped = toronto_venue_cat.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
import numpy as np #lets play with numbers

Top 10 venues

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
1,"Alderwood, Long Branch",Pizza Place,Playground,Pub,Skating Rink,Coffee Shop,Pharmacy,Gym,Sandwich Place,Medical Center,Mediterranean Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Sushi Restaurant,Frozen Yogurt Shop,Sandwich Place,Supermarket,Bridal Shop,Shopping Mall,Gas Station
3,Bayview Village,Japanese Restaurant,Bank,Café,Chinese Restaurant,Yoga Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Spa,Butcher,Café,Liquor Store,Sushi Restaurant,Juice Bar


In [43]:
#time to make some clusters
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [44]:
# set number of clusters
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

Checking the labelling of our model

In [45]:
kmeans.labels_[0:100]

array([4, 0, 4, 4, 1, 4, 4, 1, 4, 2, 4, 1, 4, 4, 4, 0, 1, 4, 4, 4, 4, 4,
       1, 4, 4, 2, 1, 4, 4, 1, 4, 4, 4, 1, 1, 4, 4, 0, 4, 4, 4, 4, 4, 0,
       4, 4, 4, 1, 4, 0, 1, 2, 1, 4, 3, 4, 4, 2, 4, 4, 1, 1, 4, 1, 4, 0,
       2, 1, 1, 2, 4, 4, 4, 0, 4, 4, 4, 4, 4, 1, 1, 4, 4, 2, 4, 2, 4, 1,
       4, 1, 0, 4, 4, 4, 0, 2, 1, 4, 2], dtype=int32)

Let's add the clustering Label column to the top 10 common venue categories

In [46]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Join toronto_grouped with combined_data on neighbourhood to add latitude & longitude for each neighborhood to prepare it for plotting

In [47]:
toronto_merged = combined_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Construction & Landscaping,Bar,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Medical Center,Rental Car Location,Donut Shop,Bank,Intersection,Restaurant,Electronics Store,Mexican Restaurant,Breakfast Spot,Miscellaneous Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Yoga Studio,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Gas Station,Bank,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


let's remove null values

In [48]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

some action Plotting the clusters on the map

In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

Let's verify each of our clusters

Cluster 1

In [51]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,0.0,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Gas Station,Noodle House,Bank
24,North York,0.0,Coffee Shop,Grocery Store,Pizza Place,Pharmacy,Supermarket,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
35,East York,0.0,Pizza Place,Gym / Fitness Center,Intersection,Flea Market,Café,Bank,Athletics & Sports,Gastropub,Breakfast Spot,Pharmacy
72,North York,0.0,Pizza Place,Sushi Restaurant,Bakery,Japanese Restaurant,Yoga Studio,Monument / Landmark,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
81,York,0.0,Convenience Store,Grocery Store,Pizza Place,Bus Line,Yoga Studio,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
89,Etobicoke,0.0,Pizza Place,Playground,Pub,Skating Rink,Coffee Shop,Pharmacy,Gym,Sandwich Place,Medical Center,Mediterranean Restaurant
96,North York,0.0,Pizza Place,Yoga Studio,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
99,Etobicoke,0.0,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Intersection,Chinese Restaurant,Sandwich Place,Discount Store,Miscellaneous Shop,Mexican Restaurant,Mobile Phone Shop
100,Etobicoke,0.0,Sandwich Place,Pizza Place,Mobile Phone Shop,Bus Line,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


Cluster 2

In [52]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,1.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Yoga Studio,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
6,Scarborough,1.0,Coffee Shop,Convenience Store,Hobby Shop,Department Store,Discount Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Modern European Restaurant
8,Scarborough,1.0,Motel,American Restaurant,Skating Rink,Yoga Studio,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
18,North York,1.0,Clothing Store,Coffee Shop,Juice Bar,Bank,Restaurant,Japanese Restaurant,Food Court,Shopping Mall,Movie Theater,Liquor Store
27,North York,1.0,Coffee Shop,Gym,Restaurant,Bus Line,Asian Restaurant,Chinese Restaurant,Sandwich Place,Beer Store,Bike Shop,Supermarket
29,North York,1.0,Caribbean Restaurant,Coffee Shop,Falafel Restaurant,Bar,Furniture / Home Store,Massage Studio,Pet Store,Performing Arts Venue,Mediterranean Restaurant,Men's Store
33,North York,1.0,Liquor Store,Grocery Store,Athletics & Sports,Coffee Shop,Yoga Studio,Miscellaneous Shop,Motel,Monument / Landmark,Modern European Restaurant,Mobile Phone Shop
34,North York,1.0,Intersection,Portuguese Restaurant,Coffee Shop,Hockey Arena,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
38,East York,1.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Furniture / Home Store,Bank,Breakfast Spot,Brewery,Sports Bar,Fish & Chips Shop,Shopping Mall
43,East Toronto,1.0,Coffee Shop,Bakery,Café,American Restaurant,Yoga Studio,Gay Bar,Cheese Shop,Comfort Food Restaurant,Coworking Space,Fish Market


Cluster 3

In [53]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,2.0,Park,Yoga Studio,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
23,North York,2.0,Convenience Store,Park,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
25,North York,2.0,Food & Drink Shop,Park,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
30,North York,2.0,Airport,Snack Place,Park,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
40,East York/East Toronto,2.0,Park,Convenience Store,Intersection,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
44,Central Toronto,2.0,Park,Swim School,Bus Line,Yoga Studio,Monument / Landmark,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
48,Central Toronto,2.0,Park,Restaurant,Tennis Court,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
50,Downtown Toronto,2.0,Park,Playground,Trail,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
74,York,2.0,Park,Women's Store,Pool,Hardware Store,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
90,Etobicoke,2.0,Pool,Park,River,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


Cluster 4

In [54]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


Cluster 5

In [55]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Construction & Landscaping,Bar,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
2,Scarborough,4.0,Medical Center,Rental Car Location,Donut Shop,Bank,Intersection,Restaurant,Electronics Store,Mexican Restaurant,Breakfast Spot,Miscellaneous Shop
4,Scarborough,4.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Gas Station,Bank,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
5,Scarborough,4.0,Playground,Jewelry Store,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
7,Scarborough,4.0,Bakery,Park,Soccer Field,Metro Station,Intersection,Ice Cream Shop,Bus Line,Mobile Phone Shop,Monument / Landmark,Modern European Restaurant
9,Scarborough,4.0,College Stadium,Skating Rink,General Entertainment,Café,Yoga Studio,Movie Theater,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
10,Scarborough,4.0,Indian Restaurant,Gaming Cafe,Chinese Restaurant,Vietnamese Restaurant,Pet Store,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Modern European Restaurant,Mobile Phone Shop
11,Scarborough,4.0,Smoke Shop,Auto Garage,Bakery,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mobile Phone Shop
12,Scarborough,4.0,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
14,Scarborough,4.0,Park,Arts & Crafts Store,Intersection,Playground,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Movie Theater


all 5 clusters added successfully